[Deep Learning Summer School 2019](http://2019.dl-lab.eu) in Gdansk, Poland  
Ordinal Regression Tutorial by [Sebastian Raschka](https://sebastianraschka.com)  
GitHub Repository: https://github.com/rasbt/DL-Gdasnk2019-tutorial  

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.8
IPython 7.2.0

torch 1.1.0


# Modifying the UTKFace DataLoader for Extended Binary Classification

## Imports

In [2]:
import time
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## The Previous Custom Dataset Class for Cross-Entropy Classification

```python
class UTKDatasetAge(Dataset):
    """Custom Dataset for loading UTKFace images"""

    def __init__(self, csv_path, img_dir, transform=None):

        df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.df = df
        self.y = df['age'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.df.iloc[index]['filename']))

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]

        return img, label

    def __len__(self):
        return self.y.shape[0]
```

## The Modfied DatasetLoader for Extended Binary Classification for Ordinal Regression

In [3]:
class UTKDatasetAgeBinary(Dataset):
    """Custom Dataset for loading UTKFace images"""

    def __init__(self, csv_path, img_dir, num_classes, transform=None):

        df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.df = df
        self.y = df['age'].values
        self.transform = transform

        ###################################
        # New:
        self.num_classes = num_classes
        ###################################

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.df.iloc[index]['filename']))

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]

        #############################################################################
        ##########################################
        ## EXERCISE: Complete the following lines of code
        ##########################################
        
        levels = [1]*label + [0]*( ### ??? 
        levels = torch.tensor(levels, dtype=torch.float32)
        #############################################################################

        return img, label, levels

    def __len__(self):
        return self.y.shape[0]

## Setting Up DataLoaders

In [4]:
TRAIN_CSV_PATH = 'training_set.csv'
TEST_CSV_PATH = 'test_set.csv'
IMAGE_PATH = 'UTKFace'

BATCH_SIZE = 256

df_train = pd.read_csv(TRAIN_CSV_PATH)
NUM_CLASSES = np.unique(df_train['age'].values).shape[0]

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
train_indices = torch.arange(0, df_train.shape[0]-1000).numpy()
valid_indices = torch.arange(df_train.shape[0]-1000, df_train.shape[0]).numpy()
del df_train


train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)



train_transform = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.RandomCrop((120, 120)),
                                      transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((128, 128)),
                                     transforms.CenterCrop((120, 120)),
                                     transforms.ToTensor()])


train_dataset = UTKDatasetAgeBinary(csv_path=TRAIN_CSV_PATH,
                                    img_dir=IMAGE_PATH,
                                    num_classes=NUM_CLASSES,
                                    transform=train_transform)

valid_dataset = UTKDatasetAgeBinary(csv_path=TRAIN_CSV_PATH,
                                    img_dir=IMAGE_PATH,
                                    num_classes=NUM_CLASSES,
                                    transform=test_transform)

test_dataset = UTKDatasetAgeBinary(csv_path=TEST_CSV_PATH,
                                   img_dir=IMAGE_PATH,
                                   num_classes=NUM_CLASSES,
                                   transform=test_transform)
  



train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          sampler=train_sampler)

valid_loader = DataLoader(valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          sampler=valid_sampler)


test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

In [6]:
# Checking the dataset
for images, labels, levels in test_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Ext. binary dimensions:', levels.shape)
    break
    
for images, labels, levels in valid_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Ext. binary dimensions:', levels.shape)
    break
    
for images, labels,levels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Ext. binary dimensions:', levels.shape)
    break

Image batch dimensions: torch.Size([256, 3, 120, 120])
Image label dimensions: torch.Size([256])
Ext. binary dimensions: torch.Size([256, 39])
Image batch dimensions: torch.Size([256, 3, 120, 120])
Image label dimensions: torch.Size([256])
Ext. binary dimensions: torch.Size([256, 39])
Image batch dimensions: torch.Size([256, 3, 120, 120])
Image label dimensions: torch.Size([256])
Ext. binary dimensions: torch.Size([256, 39])


## Iterating through the Custom Dataset

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (features, labels, levels) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', labels.size()[0])
        
        features = features.to(DEVICE)
        labels = labels.to(DEVICE)
        levels = levels.to(DEVICE)

        break

Epoch: 1 | Batch index: 0 | Batch size: 256
Epoch: 2 | Batch index: 0 | Batch size: 256


In [8]:
print(f'Labels:\n {labels}\n\n')
print(f'Levels:\n {levels}\n\n')

Labels:
 tensor([19,  3, 39,  7,  2, 29,  5, 29, 20, 24, 18, 30,  2, 24,  6, 35,  9,  4,
        34,  7, 20, 14,  7, 19, 20,  3, 28,  5, 33, 33,  4,  4, 11,  3,  4,  4,
         6, 30,  5, 25,  0,  2,  6,  5, 35, 15, 29, 14,  7,  1,  5,  3, 14,  3,
         7,  3,  4,  3,  9,  5, 39,  2, 39, 31,  7,  5, 39,  5, 24,  6,  8, 13,
        17, 14,  8,  2, 19,  4,  5, 14,  5,  5,  5, 10, 30, 24,  4, 29, 29,  5,
         1,  3,  7,  6, 19, 31, 10,  1,  2, 13,  9,  9, 13,  0, 19, 13,  4,  5,
        39, 13, 34, 26,  7, 15, 24, 11, 12,  8, 19, 23,  5,  7,  1, 11,  3,  5,
        28, 21, 10,  2,  0, 15, 24, 13, 13,  0,  5,  6,  7,  6,  2,  5, 29, 31,
         2,  5, 35, 37, 17, 13, 17,  5,  7,  5,  5,  3,  5,  5, 10,  9, 11, 14,
        16,  1,  8,  7, 27,  5,  5, 24,  3,  6,  4,  7, 14,  9, 39,  4, 30,  5,
        31, 13,  5, 18, 28, 10, 11, 14, 17, 15, 15,  9,  1,  1, 36, 34,  6, 13,
        19,  3,  7, 11, 12, 15, 11, 39,  7, 31,  7,  8,  1, 11, 14, 10, 19, 19,
         5, 25, 17, 35, 13, 15,

## Expected Loss for Random Prediction

In [11]:
-np.log(0.5) * 40

27.725887222397812